## Loading data

In [1]:
import math
import numpy as np
import pandas as pd
import sys
import re
from functions import *
from mining_functions import *
import seaborn as sns
from scipy import stats
from tabulate import tabulate
np.set_printoptions(precision=2)
pd.set_option('precision', 2)
%matplotlib inline
matplotlib.style.use('ggplot')
%load_ext autoreload
%autoreload 2

C:\Users\sperez8\AppData\Local\Continuum\Anaconda2\lib\site-packages\numpy\lib\function_base.py:4011: RuntimeWarning: Invalid value encountered in median
  r = func(a, **kwargs)


In [2]:
PATH = '/Google Drive/Jonathan Sarah Ido folder/data/CCK/'
def get_path(path = PATH):
    if os.name == 'posix':
        path = '/Documents/code/Phet-log-analyzer/cck/raw_data_parsing_check/'
        return os.environ['HOME']+path #'/Google Drive/Jonathan Sarah Ido folder/data/CCK/'
    elif os.name == 'nt':
        if os.getenv("COMPUTERNAME") == 'PYRRHA':
                path = '/Documents/git/Phet-log-analyzer/cck/raw_data_parsing_check/'
        else:
            path = '/git/Phet-log-analyzer/cck/raw_data_parsing_check/'
        return os.environ['USERPROFILE']+ path.replace('/','\\') #'\\Google Drive\Jonathan Sarah Ido folder\data\CCK\\'
    else:
        raise Exception('OS not recongnized. I\'m confused.')
        
df = pd.read_csv(get_path() + 'phet_cck_user_actions+sophistication_WITHPAUSE_more_circuit_info_with_perfect_pre.txt',index_col=False)
# dfx = pd.read_csv('C:\Users\Sarah\Documents\git\Phet-log-analyzer\cck\\raw_data_parsing_check\phet_cck_user_actions+sophistication_WITHPAUSE_more_circuit_info.txt',index_col=False)
df["student"] = df["student"].astype('category')
df["Family"]=df["Family"].str.capitalize()
df["Family_tool"]=df["Family_tool"].str.capitalize()
df["Family_default"]=df["Family_default"].str.capitalize()
df["Family_both"]=df["Family_both"].str.capitalize()

df_scores = pd.read_csv(data_path + 'MATCHING_phet_cck_user_data_anonymized_with_perfect_pre.txt')
df_scores["student"] = df_scores["student"].astype('category')

In [3]:
len(set(df['student']))

99


#### Crop activity data to get rid of activity #3

In [4]:
max_time = 25*60*1000
max_times = {s:min(df[df['student']==s]["Time Stamp"])+max_time for s in set(df['student'])}

def keep_by_time (row):
    if row["Time Stamp"] <= max_times[row["student"]]:
        return True
    else: 
        return False

df['keep'] = df.apply (lambda row: keep_by_time (row),axis=1)
df=df[df['keep']]

Functions from the notebook ""

In [5]:
def add_text(attribute,family_category,cut_off, shortest_seq_length, longest_seq_length,B):
    text = """Showing sequences for students split by {0}, using the categories {1}.
            Removed sequences used by less than {2}% of students.
            Found sequences of lenght {3} to {4}.
            Using {5} time bins""".format(attribute,family_category,int(cut_off*100), shortest_seq_length, longest_seq_length,B)
    fig = plt.figure(figsize=(10, 4))
    ax = plt.gca()
    ax.text(0.5,0.5,text,
        horizontalalignment='center',
        verticalalignment='center',
        fontsize = 15)
    plt.axis('off')
    return fig

def rank_sequences(sequence_counts,B,axesnum=None,ignore_first_time_bin=False):
    ranks = []
    for seq,counts in sequence_counts.iteritems():
#         if np.sum(counts)>0:
        ranks.append((seq,calc_infogain(counts,B,axesnum,ignore_first_time_bin)))
    return sorted(ranks, key=lambda tup: tup[1])

def get_top_seqs(ranks,N):
    return ranks[-N:]

## Preparing our groups
So we want to take a look at what the high prior knowledge are doing. Remember that we have 25 such students, 22 HH and 3 HL.

In [6]:
median_learning0 = 0.45
df_scores['split pre'] = df_scores.apply (lambda row: label_learning (median_learning0,row,"z pre"),axis=1)
median_learning2 = np.median(df_scores[df_scores['split pre']=='low']['z post t2'])
df_scores['split post t2'] = df_scores.apply (lambda row: label_learning (median_learning2,row,"z post t2"),axis=1)

In [7]:
for x in ['high','low']:
    for y in ['high','low']:
        print x,y, len(set(df_scores[(df_scores['split pre']==x)&(df_scores['split post t2']==y)]['student']))

high high 22
high low 3
low high 38
low low 36


We want to compare the three groups to each other. We would like to know if:
- HH look like LL in which case LL use experts strategies that are productive with certain level of prior knowledge
- HH look like LH in which case they used productive strategies

We need to make a new funciton to run the sequence mining with 3 groups.

### Let's make those three groups

In [8]:
def label_3_groups (row):
    if row['split pre']  == 'high' and row['split post t2'] == 'high':
        return 'HH'
    if row['split pre']  == 'high' and row['split post t2'] == 'low':
        return 'HL'
    if row['split pre']  == 'low' and row['split post t2'] == 'high':
        return 'LH'
    if row['split pre']  == 'low' and row['split post t2'] == 'low':
        return 'LL'
df_scores['three groups'] = df_scores.apply (lambda row: label_3_groups (row),axis=1)

## Let's run the sequence mining like we usually do on all 3 groups

In [9]:
# students = list(set(get_students()) - set(df_scores[df_scores['three groups']=='HL']['student']))
students = list(set(df_scores['student']) - set(df_scores[df_scores['three groups']=='HL']['student']))
CUT_OFF = 0.40 #we keep only sequences used once by at least 10% of students
shortest_seq_length = 1
longest_seq_length = 10

### PARAMETERS
attribute = 'three groups'
level1 = 'LH'
level2 = 'LL'
level3 = 'HH'
family_category = "Family"
group_sizes = np.array([len(set(df_scores[df_scores['three groups']==level1]['student'])),len(set(df_scores[df_scores['three groups']==level2]['student'])),len(set(df_scores[df_scores['three groups']==level3]['student']))])

BINS = 1
axis = 1
sequence_counts_group = get_sequence_use_by_timebin(df,students,family_category,
                                          BINS,attribute,level1,level2,
                                          shortest_seq_length,longest_seq_length,CUT_OFF,level3='HH',ignore=['I'])
sequence_counts_group = {s:(counts.T/group_sizes).T for s,counts in sequence_counts_group.iteritems()}
tops_group = get_top_seqs(rank_sequences(sequence_counts_group,BINS,axis),15)

BINS = 3
axis = None
sequence_counts_both = get_sequence_use_by_timebin(df,students,family_category,
                                  BINS,attribute,level1,level2,
                                  shortest_seq_length,longest_seq_length,CUT_OFF,level3='HH',ignore=['I'])
sequence_counts_both = {s:counts/group_sizes[:,None] for s,counts in sequence_counts_both.iteritems()}
tops_both = get_top_seqs(rank_sequences(sequence_counts_both,BINS,axis),15)

axis = 0
tops_time = get_top_seqs(rank_sequences(sequence_counts_both,BINS,axis),15)

Getting sequence use over 1 time bins for 96 students split by three groups. 
    Keeping only sequences used once by at least 40% of students 
    in each group and overall.
Getting sequence use over 3 time bins for 96 students split by three groups. 
    Keeping only sequences used once by at least 40% of students 
    in each group and overall.


In [10]:
table = [['SEQ','IG','LH','LL','HH','chi^2','p-val', 'LH-res', "LL-res", 'HH-res']]
tops_group.sort(key=lambda x: -x[1])
for seq,ig in tops_group:
#     print seq, round(ig,3), np.ndarray.tolist(sequence_counts_group[seq].T*group_sizes)[0]
    lh,ll,hh = (sequence_counts_group[seq].T*group_sizes)[0]
    tb = [[lh, group_sizes[0]-lh], [ll, group_sizes[1]-ll], [hh, group_sizes[2]-hh]]
    chi2, p, dof, ex = stats.chi2_contingency(tb, correction=False)
    res = (tb - ex)/np.sqrt(ex)
    cmarg = np.array(tb).sum(axis=0)
    rmarg = np.array(tb).sum(axis=1)
    n = np.array(tb).sum()
    res_adj = res
    res_adj[:,0] = res_adj[:,0]/np.sqrt(1-rmarg/n)/np.sqrt(1-cmarg[0]/n)
    row = [seq, str(round(ig,2))] + [int(x*100) for x in list(np.ndarray.tolist(sequence_counts_group[seq].T)[0])] + [round(chi2, 3), round(p, 3)] + [round(r,2) for r in res_adj[:,0]]
    table.append(row)
print tabulate(table)
np.savetxt("by_group_threshold30.csv", table, delimiter=",", fmt='%s')

-------  ----  --  --  --  ------  -----  ------  ------  ------
SEQ      IG    LH  LL  HH  chi^2   p-val  LH-res  LL-res  HH-res
PTcCTc   0.15  47  33  13  7.056   0.029  2.17    -0.17   -2.33
CTsCTs   0.15  26  44  13  6.599   0.037  -0.67   2.35    -1.93
PCTbP    0.11  21  16  40  4.696   0.096  -0.54   -1.3    2.12
PTsPC    0.11  52  19  45  9.172   0.01   2.3     -2.98   0.76
TbPCP    0.1   23  16  40  4.338   0.114  -0.24   -1.46   1.96
PTsP     0.09  60  25  54  10.271  0.006  2.34    -3.17   0.93
TcCPC    0.09  21  50  31  6.95    0.031  -2.22   2.5     -0.29
CPCTs    0.08  26  41  18  4.024   0.134  -0.67   1.89    -1.4
CPTs     0.08  39  19  45  5.224   0.073  1.03    -2.24   1.37
CPCPCPC  0.08  18  38  40  4.844   0.089  -2.2    1.25    1.11
CPCTcC   0.07  28  52  27  5.753   0.056  -1.4    2.4     -1.13
PTcC     0.06  52  55  27  4.937   0.085  0.75    1.16    -2.21
CTcCTcP  0.06  39  47  22  3.482   0.175  0.15    1.35    -1.74
CTsPC    0.06  44  44  22  3.391   0.183  0.8

In [11]:
table = [['SEQ','IG','t1','t2','t3', 't4']]
tops_time.sort(key=lambda x: -x[1])
for seq,ig in tops_time:
#     print seq, round(ig,3), np.ndarray.tolist(sequence_counts_time[seq].T*group_sizes)[0]
    row = [seq,str(round(ig,2))] + [int(x*100) for x in np.sum((sequence_counts_both[seq].T*group_sizes).T, axis=0)/sum(group_sizes)]
    table.append(row)
print tabulate(table)

-------  ----  --  --  --
SEQ      IG    t1  t2  t3
TbCP     0.74  32  3   4
PTb      0.72  40  6   3
CTbP     0.49  42  9   7
TbPC     0.43  42  9   9
CTbPC    0.4   31  8   6
CTsPTs   0.4   29  16  2
TbP      0.38  50  11  12
PTsPC    0.38  28  11  4
PCTbP    0.37  16  3   6
PCTsP    0.36  26  20  2
CPCTs    0.32  23  9   5
TbPCP    0.3   18  6   4
CTcPTcP  0.3   11  25  4
TcPTcPC  0.29  8   27  7
CTbC     0.28  42  11  14
-------  ----  --  --  --


In [12]:
table = [['SEQ','IG','LH','LL','HH','t1','t2','t3','t4']]
tops_both.sort(key=lambda x: -x[1])
for seq,ig in tops_both:
#     print seq, round(ig,3), np.ndarray.tolist(sequence_counts_both[seq].T*group_sizes)[0]
    row = [seq,str(round(ig,2))] + [int(x*100) for x in np.sum(sequence_counts_both[seq], axis=1)] + [int(x*100) for x in np.sum((sequence_counts_both[seq].T*group_sizes).T, axis=0)/sum(group_sizes)]
    table.append(row)
print tabulate(table)

-------  ----  --  --  --  --  --  --
SEQ      IG    LH  LL  HH  t1  t2  t3
TbCP     0.82  47  33  36  32  3   4
PTb      0.81  52  38  63  40  6   3
CPCTs    0.57  36  52  18  23  9   5
PTsPC    0.56  57  22  54  28  11  4
PCTbP    0.56  21  19  45  16  3   6
CTbP     0.54  50  58  77  42  9   7
TbPC     0.46  63  55  68  42  9   9
CTsPTs   0.45  52  38  54  29  16  2
CTbPC    0.44  39  44  59  31  8   6
PCTsP    0.44  63  36  45  26  20  2
TbP      0.42  71  66  90  50  11  12
TbPCP    0.38  28  22  40  18  6   4
CTsCTs   0.37  34  55  18  18  14  5
CTcPTcP  0.36  47  41  27  11  25  4
PTsP     0.36  73  27  68  31  16  7
-------  ----  --  --  --  --  --  --


## ignore first time bin

In [20]:
def get_frequencies_without_first_time_bin(frequencies_by_bin,students_in_group):
    '''
    Takes frequencies of sequencies by time bins and removes the first time bin.
    Then sums the frequency per sequence per student overall.
    Return this information for only relevant students
    
    Arguments:
        frequencies_by_bin = {student: [list of Counters for each bin]}
                           = {student1: [ Counter{'TPT':3, 'CPT':5...}, Counter{},... ],  ...}
        students_in_group = list of students for whom to return this data
        
    returns:
        {student1: Counter{'TPT':5, ...},  ...}
    '''
    return {student:sum(bins[1:], Counter()) for student,bins in frequencies_by_bin.iteritems() if student in students_in_group}


def get_sequence_use_by_timebin(df, students, category_column, B, attribute,
            level1, level2, shortest_seq_length, longest_seq_length, cut_off,level3 = None,
            remove_actions = [],ignore=['I'],ignore_first_time_bin=False):
    '''
    '''
    
    print """Getting sequence use over {3} time bins for {0} students split by {1}. 
    Keeping only sequences used once by at least {2}% of students 
    in each group and overall.""".format(len(students),attribute,int(cut_off*100),B)

    #get all seqs per student per time bin        
    blocks, time_coords =  get_blocks_withTime_new(df, students, category_column, start=False, ignore=ignore, remove_actions = remove_actions)
    frequencies_by_bin = get_frequencies_by_bin(blocks, students, time_coords, B, shortest = shortest_seq_length, longest = longest_seq_length)

    cleaned_frequencies = Counter()
    levels = [level1,level2]
    if level3: levels.append(level3)
    for level in levels:
        students_in_group = get_students(attribute,level)
        N = int(math.ceil(cut_off*len(students_in_group)))
#         print level
        
#         students_in_group = [19930112]
        blocks, time_coords =  get_blocks_withTime_new(df, students_in_group, category_column, start=False, ignore=ignore, remove_actions = remove_actions)
        #find all sequences to consider for analysis, given that they have been used by enough students
        if ignore_first_time_bin:
            frequencies =  get_frequencies_without_first_time_bin(frequencies_by_bin,students_in_group)
        else:
            frequencies = get_frequencies(blocks, shortest = shortest_seq_length, longest = longest_seq_length)
#         frequencies[19930112]['C']=100
#         print frequencies[19930112]
        ## number of students who used freq Counter{'TPT':5,...}
        counts_frequencies = Counter({f:sum([ 1 if f in freq else 0 for freq in frequencies.values()]) for f in list(sum(frequencies.values(),Counter()))})
        cleaned_frequencies += remove_rare_frequencies(counts_frequencies, N)    
    
    counts = count_use_per_group_per_bin(cleaned_frequencies, frequencies_by_bin, B, attribute, level1, level2, level3 = level3)
    return counts

In [21]:
get_sequence_use_by_timebin(df,students[0:1],family_category,
                                  BINS,attribute,level1,level2,
                                  shortest_seq_length,longest_seq_length,CUT_OFF,
                                    level3='HH',ignore=['I'],ignore_first_time_bin=True)

Getting sequence use over 3 time bins for 1 students split by three groups. 
    Keeping only sequences used once by at least 40% of students 
    in each group and overall.


{}

In [22]:
get_sequence_use_by_timebin(df,students[0:1],family_category,
                                  BINS,attribute,level1,level2,
                                  shortest_seq_length,longest_seq_length,CUT_OFF,
                                    level3='HH',ignore=['I'],ignore_first_time_bin=False)

Getting sequence use over 3 time bins for 1 students split by three groups. 
    Keeping only sequences used once by at least 40% of students 
    in each group and overall.


{'C': array([[ 0.,  0.,  0.],
        [ 1.,  1.,  1.],
        [ 0.,  0.,  0.]]), 'CP': array([[ 0.,  0.,  0.],
        [ 1.,  1.,  1.],
        [ 0.,  0.,  0.]]), 'CPC': array([[ 0.,  0.,  0.],
        [ 1.,  1.,  1.],
        [ 0.,  0.,  0.]]), 'CPCP': array([[ 0.,  0.,  0.],
        [ 1.,  1.,  1.],
        [ 0.,  0.,  0.]]), 'CPCPC': array([[ 0.,  0.,  0.],
        [ 1.,  1.,  1.],
        [ 0.,  0.,  0.]]), 'CPCPCP': array([[ 0.,  0.,  0.],
        [ 1.,  0.,  1.],
        [ 0.,  0.,  0.]]), 'CPCPCPC': array([[ 0.,  0.,  0.],
        [ 1.,  0.,  1.],
        [ 0.,  0.,  0.]]), 'CPCTc': array([[ 0.,  0.,  0.],
        [ 0.,  1.,  0.],
        [ 0.,  0.,  0.]]), 'CPCTcC': array([[ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.]]), 'CPCTs': array([[ 0.,  0.,  0.],
        [ 1.,  0.,  0.],
        [ 0.,  0.,  0.]]), 'CPTc': array([[ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.]]), 'CPTs': array([[ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0., 

In [23]:
%aimport mining_functions
# students = list(set(get_students()) - set(df_scores[df_scores['three groups']=='HL']['student']))
students = list(set(df_scores['student']) - set(df_scores[df_scores['three groups']=='HL']['student']))
CUT_OFF = 0.40 #we keep only sequences used once by at least 10% of students
shortest_seq_length = 1
longest_seq_length = 10

### PARAMETERS
attribute = 'three groups'
level1 = 'LH'
level2 = 'LL'
level3 = 'HH'
family_category = "Family"
group_sizes = np.array([len(set(df_scores[df_scores['three groups']==level1]['student'])),len(set(df_scores[df_scores['three groups']==level2]['student'])),len(set(df_scores[df_scores['three groups']==level3]['student']))])

# BINS = 1
# axis = 1
# sequence_counts_group = get_sequence_use_by_timebin(df,students,family_category,
#                                           BINS,attribute,level1,level2,
#                                           shortest_seq_length,longest_seq_length,CUT_OFF,level3='HH',ignore=['I'])
# sequence_counts_group = {s:(counts.T/group_sizes).T for s,counts in sequence_counts_group.iteritems()}
# tops_group = get_top_seqs(rank_sequences(sequence_counts_group,BINS,axis),15)

BINS = 3
axis = None
sequence_counts_both_ignore_first = get_sequence_use_by_timebin(df,students,family_category,
                                  BINS,attribute,level1,level2,
                                  shortest_seq_length,longest_seq_length,CUT_OFF,
                                    level3='HH',ignore=['I'],ignore_first_time_bin=True)
sequence_counts_both_ignore_first = {s:counts/group_sizes[:,None] for s,counts in sequence_counts_both_ignore_first.iteritems()}
tops_both_ignore_first = get_top_seqs(rank_sequences(sequence_counts_both_ignore_first,BINS,axis),15)

axis = 0
tops_time_ignore_first = get_top_seqs(rank_sequences(sequence_counts_both_ignore_first,BINS,axis,ignore_first_time_bin=True),15)

Getting sequence use over 3 time bins for 96 students split by three groups. 
    Keeping only sequences used once by at least 40% of students 
    in each group and overall.


## cstg

In [24]:
table = [['SEQ','IG','t1','t2','t3', 't4']]
tops_time_ignore_first.sort(key=lambda x: -x[1])
for seq,ig in tops_time_ignore_first:
#     print seq, round(ig,3), np.ndarray.tolist(sequence_counts_time[seq].T*group_sizes)[0]
    row = [seq,str(round(ig,2))] + [int(x*100) for x in np.sum((sequence_counts_both_ignore_first[seq].T*group_sizes).T, axis=0)/sum(group_sizes)]
    table.append(row)
print tabulate(table)

-------  ----  --  --  --
SEQ      IG    t1  t2  t3
TcPTcP   0.22  10  30  9
PTs      0.15  41  27  10
CTcPTc   0.14  20  32  12
TsPC     0.13  41  30  13
TcPTc    0.11  19  44  20
TsP      0.09  47  35  17
TcPCTc   0.07  14  34  15
PCTcCTc  0.07  27  19  10
CPCTc    0.07  33  33  16
PTcP     0.06  14  40  21
CTcCTcC  0.06  17  21  13
TsC      0.05  40  30  18
PCTcP    0.05  25  33  19
PTcPC    0.04  10  33  19
CPTc     0.04  23  29  18
-------  ----  --  --  --


In [31]:
table = [['SEQ','IG','LH','LL','HH','t1','t2','t3','t4']]
tops_both_ignore_first.sort(key=lambda x: -x[1])
for seq,ig in tops_both_ignore_first:
#     print seq, round(ig,3), np.ndarray.tolist(sequence_counts_both[seq].T*group_sizes)[0]
#     print seq,sequence_counts_both_ignore_first[seq]
    row = [seq,str(round(ig,2))] + [int(x*100) for x in np.sum(sequence_counts_group[seq], axis=1)] + [int(x*100) for x in np.sum((sequence_counts_both_ignore_first[seq].T*group_sizes).T, axis=0)/sum(group_sizes)]
    table.append(row)
print tabulate(table)

-------  ----  --  --  --  --  --  --
SEQ      IG    LH  LL  HH  t1  t2  t3
CTcCTcC  0.34  28  47  40  17  21  13
TcCPC    0.32  21  50  31  10  18  20
TcPTcP   0.31  50  41  31  10  30  9
PCTcCTc  0.25  31  55  40  27  19  10
CTcCTc   0.25  50  66  50  36  30  21
CTb      0.23  71  58  81  60  23  22
PTs      0.22  65  44  63  41  27  10
TcPCTc   0.2   47  44  40  14  34  15
TcPTc    0.2   63  61  40  19  44  20
PTcP     0.19  63  50  45  14  40  21
PTcPC    0.19  52  47  45  10  33  19
Tb       0.19  81  63  90  68  28  27
TsPC     0.16  68  55  54  41  30  13
CTcPTc   0.16  52  52  36  20  32  12
CPCPCP   0.15  26  52  50  26  17  18
-------  ----  --  --  --  --  --  --


In [ ]:
sns.set_style("darkgrid")
sns.set_context("paper")
all_seqs = zip(*tops_both_ignore_first)[0]
fig, axes = plt.subplots(len(tops_both_ignore_first),1)#, sharex=True, sharey=True)
maximum = max([np.amax(counts) for seq,counts in sequence_counts_both_ignore_first.iteritems() if seq in all_seqs])
for i,(seq,ig) in enumerate(tops_both_ignore_first):
    ax = axes[i]
    data = sequence_counts_both_ignore_first[seq]
    hl,ll,hh = np.ndarray.tolist(sequence_counts_group[seq].T*group_sizes)[0]
    seq = "$"+seq.replace('Ts','T_2').replace('Tb','T_1').replace('Tc','T_m')+"$"
    ax.text(0,3.05, seq, fontsize=14)
    ax.text(2.0,3.05, "$IG$= {0}, $LH$: {1}, $LL$: {2}, $HH$: {3}".format(str(round(ig,2)),hl,ll,hh), fontsize=10)
    
    heatmap = sns.heatmap(data, ax=ax, cmap=plt.cm.Blues,alpha=0.8, vmin=0, vmax=maximum, cbar=False,
                          annot=True) #, annot_kws={'fontweight':'bold'})
    fig.set_size_inches(10,40)
    ax.set_frame_on(False)
    ax.set_yticks(np.arange(data.shape[0]) + 0.5)
    ylabels = ['$HH$','$LL$','$LH$']
    ax.set_yticklabels(ylabels)
    ax.set_xticklabels(['','','',''])
        
    i += 1

### Specific sequence look-up

In [ ]:
all_actions = [('Tb',0),('Ts',0),('Tc',0),('P',0),('C',0)]
table = [['SEQ','IG','LH','LL','HH','chi^2','p-val', 'LH-res', "LL-res", 'HH-res']]

seq = 'Tc'
lh,ll,hh = (sequence_counts_group[seq].T*group_sizes)[0]
tb = [[lh, group_sizes[0]-lh], [ll, group_sizes[1]-ll], [hh, group_sizes[2]-hh]]
chi2, p, dof, ex = stats.chi2_contingency(tb, correction=False)
res = (tb - ex)/np.sqrt(ex)
cmarg = np.array(tb).sum(axis=0)
rmarg = np.array(tb).sum(axis=1)
res_adj = res
res_adj[:,0] = res_adj[:,0]/np.sqrt(1-rmarg/n)/np.sqrt(1-cmarg[0]/n)
n = np.array(tb).sum()
row = [seq, str(round(ig,2))] + [int(x*100) for x in list(np.ndarray.tolist(sequence_counts_group[seq].T)[0])] + [round(chi2, 3), round(p, 3)] + [round(r,2) for r in res_adj[:,0]]
table.append(row)
print tabulate(table)

### Single actions:

In [ ]:
all_actions = [('Tb',0),('Ts',0),('Tc',0),('P',0),('C',0),]
sns.set_style("darkgrid")
sns.set_context("paper")
all_seqs = zip(*all_actions)[0]
fig, axes = plt.subplots(len(all_actions),1)#, sharex=True, sharey=True)
maximum = max([np.amax(counts) for seq,counts in sequence_counts_both.iteritems() if seq in all_seqs])

for i,(seq,ig) in enumerate(all_actions):
    ax = axes[i]
    data = sequence_counts_both[seq]
    ax = axes[i]
    hl,ll,h_ = np.ndarray.tolist(sequence_counts_group[seq].T*group_sizes)[0]
    seq = "$"+seq.replace('Ts','T_2').replace('Tb','T_1').replace('Tc','T_m')+"$"
    ax.text(0,3.05, seq, fontsize=14)
    ax.text(2.0,3.05, "$IG$= {0}, $LH$: {1}, $LL$: {2}, $HH$: {3}".format(str(round(ig,2)),hl,ll,h_), fontsize=10)
    
    heatmap = sns.heatmap(data, ax=ax, cmap=plt.cm.Blues,alpha=0.8, vmin=0, vmax=maximum, cbar=False,
                          annot=True) #, annot_kws={'fontweight':'bold'})
    fig.set_size_inches(10,12)
    ax.set_frame_on(False)
    ax.set_yticks(np.arange(data.shape[0]) + 0.5)
    ylabels = ['$HH$','$LL$','$LH$']
    ax.set_yticklabels(ylabels)
    ax.set_xticklabels(['','','',''])
        
    i += 1

### Let's analyze the building blocks

In [ ]:
blocks_LH, time_coords_LH =  get_blocks_withTime_new(df,get_students('three groups','LH'),"Family",start=False,ignore=['I'], as_list=True)
blocks_LL, time_coords_LL =  get_blocks_withTime_new(df,get_students('three groups','LL'),"Family",start=False,ignore=['I'], as_list=True)
blocks_HH, time_coords_HH =  get_blocks_withTime_new(df,get_students('three groups','HH'),"Family",start=False,ignore=['I'], as_list=True)

In [ ]:
def get_frequencies(seq_of_interest,blocks1,blocks2,blocks3,normalize_by='length'):
    def coo(seq, seq_of_interest):
        return float(occurrences(''.join(seq),seq_of_interest))
    
    if normalize_by == None:
        # count number of occurrences of "seq_of_interest" in a student's seq
        freq1 = {student : coo(seq, seq_of_interest) for student,seq in blocks1.iteritems()}
        freq2 = {student : coo(seq, seq_of_interest) for student,seq in blocks2.iteritems()}
        freq3 = {student : coo(seq, seq_of_interest) for student,seq in blocks3.iteritems()}
    elif normalize_by == 'length':
        len_seqi = len(''.join([c for c in seq_of_interest if c.isupper()]))
        # count number of occurrences of "seq_of_interest" in a student's seq divided by the length of seq
        freq1 = {student : coo(seq, seq_of_interest)/(len(seq)-len_seqi-1) if coo(seq, seq_of_interest) >0 else 0 for student,seq in blocks1.iteritems()}
        freq2 = {student : coo(seq, seq_of_interest)/(len(seq)-len_seqi-1) if coo(seq, seq_of_interest) >0 else 0 for student,seq in blocks2.iteritems()}
        freq3 = {student : coo(seq, seq_of_interest)/(len(seq)-len_seqi-1) if coo(seq, seq_of_interest) >0 else 0 for student,seq in blocks3.iteritems()}
    else:
        # count number of occurrences of "seq_of_interest" in a student's seq divided by the number of occurrences of the seq "normalize_by"
        freq1 = {student : coo(seq, seq_of_interest)/coo(seq, normalize_by) if coo(seq, normalize_by) >0 else 0 for student,seq in blocks1.iteritems()}
        freq2 = {student : coo(seq, seq_of_interest)/coo(seq, normalize_by) if coo(seq, normalize_by) >0 else 0 for student,seq in blocks2.iteritems()}
        freq3 = {student : coo(seq, seq_of_interest)/coo(seq, normalize_by) if coo(seq, normalize_by) >0 else 0 for student,seq in blocks3.iteritems()}
    return freq1.values(),freq2.values(),freq3.values()

def occurrences(string, sub):
    count = start = 0
    while True:
        start = string.find(sub, start) + 1
        if start > 0:
            count+=1
        else:
            return count
        
def make_hist_continuous(seq_of_interest,freq_LH,freq_LL,freq_HH):
    fig, axs = plt.subplots(1,figsize=(12,2.5))
    alldata = np.concatenate((freq_LH, freq_LL, freq_HH), axis=0)
    binsize = max(alldata)/10.
    bins = np.arange(0,max(alldata)+binsize,binsize)
#     plt.hist(freq_HL,color ='r',alpha = 0.4,bins = bins)
#     plt.hist(freq_LL,color = 'b',alpha = 0.4,bins = bins)
#     plt.hist(freq_HH,color = 'g',alpha = 0.4,bins = bins)
    plt.hist([freq_LH,freq_LL,freq_HH], bins=bins, stacked=True)
    plt.title("Use of the sequence ''{0}'' by LH (red) and LL (blue) and HH (purple)".format(seq_of_interest))
    return plt

In [ ]:
seq = 'Tb'
freq_LH,freq_LL,freq_HH = get_frequencies(seq,blocks_LH,blocks_LL,blocks_HH,normalize_by=None)
plot = make_hist_continuous(seq,freq_LH,freq_LL,freq_HH)
# plot.show()
# perform_comparative_stats(seq,freq_HL,freq_LL)
# freq_HL,freq_LL = get_frequencies(seq,blocks_merged_HL,blocks_merged_LL,normalize_by='P')
# plot = make_hist_continuous(seq,freq_HL,freq_LL)
# plot.show()
# perform_comparative_stats(seq,freq_HL,freq_LL)

In [ ]:
from itertools import combinations
from statsmodels.stats.multitest import multipletests
from statsmodels.stats.libqsturng import psturng
import warnings


def kw_dunn(groups, to_compare=None, alpha=0.05, method='bonf'):
    """

    Kruskal-Wallis 1-way ANOVA with Dunn's multiple comparison test

    Arguments:
    ---------------
    groups: sequence
        arrays corresponding to k mutually independent samples from
        continuous populations

    to_compare: sequence
        tuples specifying the indices of pairs of groups to compare, e.g.
        [(0, 1), (0, 2)] would compare group 0 with 1 & 2. by default, all
        possible pairwise comparisons between groups are performed.

    alpha: float
        family-wise error rate used for correcting for multiple comparisons
        (see statsmodels.stats.multitest.multipletests for details)

    method: string
        method used to adjust p-values to account for multiple corrections (see
        statsmodels.stats.multitest.multipletests for options)

    Returns:
    ---------------
    H: float
        Kruskal-Wallis H-statistic

    p_omnibus: float
        p-value corresponding to the global null hypothesis that the medians of
        the groups are all equal

    Z_pairs: float array
        Z-scores computed for the absolute difference in mean ranks for each
        pairwise comparison

    p_corrected: float array
        corrected p-values for each pairwise comparison, corresponding to the
        null hypothesis that the pair of groups has equal medians. note that
        these are only meaningful if the global null hypothesis is rejected.

    reject: bool array
        True for pairs where the null hypothesis can be rejected for the given
        alpha

    Reference:
    ---------------
    Gibbons, J. D., & Chakraborti, S. (2011). Nonparametric Statistical
    Inference (5th ed., pp. 353-357). Boca Raton, FL: Chapman & Hall.

    """

    # omnibus test (K-W ANOVA)
    # -------------------------------------------------------------------------

    groups = [np.array(gg) for gg in groups]

    k = len(groups)

    n = np.array([len(gg) for gg in groups])
    if np.any(n < 5):
        warnings.warn("Sample sizes < 5 are not recommended (K-W test assumes "
                      "a chi square distribution)")

    allgroups = np.concatenate(groups)
    N = len(allgroups)
    ranked = stats.rankdata(allgroups)

    # correction factor for ties
    T = stats.tiecorrect(ranked)
    if T == 0:
        raise ValueError('All numbers are identical in kruskal')

    # sum of ranks for each group
    j = np.insert(np.cumsum(n), 0, 0)
    R = np.empty(k, dtype=np.float)
    for ii in range(k):
        R[ii] = ranked[j[ii]:j[ii + 1]].sum()

    # the Kruskal-Wallis H-statistic
    H = (12. / (N * (N + 1.))) * ((R ** 2.) / n).sum() - 3 * (N + 1)

    # apply correction factor for ties
    H /= T

    df_omnibus = k - 1
    p_omnibus = stats.chisqprob(H, df_omnibus)

    # multiple comparisons
    # -------------------------------------------------------------------------

    # by default we compare every possible pair of groups
    if to_compare is None:
        to_compare = tuple(combinations(range(k), 2))

    ncomp = len(to_compare)

    Z_pairs = np.empty(ncomp, dtype=np.float)
    p_uncorrected = np.empty(ncomp, dtype=np.float)
    Rmean = R / n

    for pp, (ii, jj) in enumerate(to_compare):
        
        # standardized score
        ts3_ts = list(np.unique(allgroups, return_counts=True)[1])
        E_ts3_ts = sum([x**3 - x for x in ts3_ts if x>1])

        if sum([x>1 for x in ts3_ts]) > 0:
            warnings.warn("We see ties.")

            yi = np.abs(Rmean[ii] - Rmean[jj])
            theta10 = (N * (N + 1)) / 12
            theta11 =  E_ts3_ts / ( 12* (N - 1) )
            theta2 = (1. / n[ii] + 1. / n[jj])
            theta = np.sqrt( (theta10 - theta11) * theta2 )
            Zij = yi / theta
        else:
            Zij = (np.abs(Rmean[ii] - Rmean[jj]) /
                   np.sqrt((1. / 12.) * N * (N + 1) * (1. / n[ii] + 1. / n[jj])))
        
#         # standardized score
#         Zij = (np.abs(Rmean[ii] - Rmean[jj]) /
#                np.sqrt((1. / 12.) * N * (N + 1) * (1. / n[ii] + 1. / n[jj])))
        Z_pairs[pp] = Zij

    # corresponding p-values obtained from upper quantiles of the standard
    # normal distribution
    p_uncorrected = stats.norm.sf(Z_pairs) * 2.

    # correction for multiple comparisons
    reject, p_corrected, alphac_sidak, alphac_bonf = multipletests(
        p_uncorrected, method=method
    )

    return H, p_omnibus, Z_pairs, p_corrected, reject

In [ ]:
seq = 'TsC'
# norm = 'length'
norm = 'Ts'
freq_LH,freq_LL,freq_HH = get_frequencies(seq,blocks_LH,blocks_LL,blocks_HH,normalize_by=norm)
freqname = 'Frequency'
groupname = 'Group'

df0 = pd.DataFrame(freq_LH, columns=[freqname])
df0[groupname] = 'LH'
df1 = pd.DataFrame(freq_LL, columns=[freqname])
df1[groupname] = 'LL'
df2 = pd.DataFrame(freq_HH, columns=[freqname])
df2[groupname] = 'HH'
df_all = df0.append(df1).append(df2).reset_index()
# df_all.boxplot('freq',by='group')

ax = sns.boxplot(x=groupname, y=freqname, data=df_all)
ax = sns.swarmplot(x=groupname, y=freqname, data=df_all, edgecolor="black", linewidth=.9)

# F, p = stats.kruskal(freq_LH,freq_LL,freq_HH)
# print F, p
print "Comparing the median frequency usage of sequence {0} normalized by {1}".format(seq, norm)
H, p_omnibus, Z_pairs, p_corrected, reject = kw_dunn([freq_LH,freq_LL,freq_HH])
print "Kruskal omnibus test: H statistic = {0}, p<{1}".format(H, p_omnibus)
print "Dunn's pairwise post hoc test with Bonferonni:"
print "LH vs LL: Z score = {0}, p<{1}".format(Z_pairs[0], p_corrected[0])
print "LH vs HH: Z score = {0}, p<{1}".format(Z_pairs[1], p_corrected[1])
print "LL vs HH: Z score = {0}, p<{1}".format(Z_pairs[2], p_corrected[2])


## Repeat sequence analysis ignoring the first time bin

In [ ]:
%aimport mining_functions

BINS = 3
axis = None
tops_both_nofirsttimebin = get_top_seqs(rank_sequences(sequence_counts_both,BINS,axis,ignore_first_time_bin=True),15)

axis = 0
tops_time_nofirsttimebin = get_top_seqs(rank_sequences(sequence_counts_both,BINS,axis,ignore_first_time_bin=True),15)

### By group and time

In [ ]:
table = [['SEQ','IG','LH','LL','HH','t1','t2','t3','t4']]
tops_both_nofirsttimebin.sort(key=lambda x: -x[1])
for seq,ig in tops_both_nofirsttimebin:
#     print seq, round(ig,3), np.ndarray.tolist(sequence_counts_both[seq].T*group_sizes)[0]
    row = [seq,str(round(ig,2))] + [int(x*100) for x in np.sum(sequence_counts_both[seq][:,1:], axis=1)] +['-']+ [int(x*100) for x in np.sum((sequence_counts_both[seq][:,1:].T*group_sizes).T, axis=0)/sum(group_sizes)]
    table.append(row)
print tabulate(table)

In [ ]:
print "New seqs from dropping first time bin:",set(zip(*tops_both_nofirsttimebin)[0]) - set(zip(*tops_both)[0])
print "Lost seqs from dropping first time bin:", set(zip(*tops_both)[0]) - set(zip(*tops_both_nofirsttimebin)[0])

In [ ]:
sns.set_style("darkgrid")
sns.set_context("paper")
all_seqs = zip(*tops_both_nofirsttimebin)[0]
fig, axes = plt.subplots(len(tops_both_nofirsttimebin),1)#, sharex=True, sharey=True)
maximum = max([np.amax(counts) for seq,counts in sequence_counts_both.iteritems() if seq in all_seqs])
for i,(seq,ig) in enumerate(tops_both_nofirsttimebin):
    ax = axes[i]
    data = sequence_counts_both[seq]
    hl,ll,hh = np.ndarray.tolist(sequence_counts_group[seq].T*group_sizes)[0]
    seq = "$"+seq.replace('Ts','T_2').replace('Tb','T_1').replace('Tc','T_m')+"$"
    ax.text(0,3.05, seq, fontsize=14)
    ax.text(2.0,3.05, "$IG$= {0}, $LH$: {1}, $LL$: {2}, $HH$: {3}".format(str(round(ig,2)),hl,ll,hh), fontsize=10)
    
    heatmap = sns.heatmap(data, ax=ax, cmap=plt.cm.Blues,alpha=0.8, vmin=0, vmax=maximum, cbar=False,
                          annot=True) #, annot_kws={'fontweight':'bold'})
    fig.set_size_inches(10,40)
    ax.set_frame_on(False)
    ax.set_yticks(np.arange(data.shape[0]) + 0.5)
    ylabels = ['$HH$','$LL$','$LH$']
    ax.set_yticklabels(ylabels)
    ax.set_xticklabels(['','','',''])
        
    i += 1

### by time

In [ ]:
print "New seqs from dropping first time bin:",set(zip(*tops_time_nofirsttimebin)[0]) - set(zip(*tops_time)[0])
print "Lost seqs from dropping first time bin:", set(zip(*tops_time)[0]) - set(zip(*tops_time_nofirsttimebin)[0])

In [ ]:
sns.set_style("darkgrid")
sns.set_context("paper")
all_seqs = zip(*tops_time_nofirsttimebin)[0]
fig, axes = plt.subplots(len(tops_time_nofirsttimebin),1)#, sharex=True, sharey=True)
maximum = max([np.amax(counts) for seq,counts in sequence_counts_both.iteritems() if seq in all_seqs])
for i,(seq,ig) in enumerate(tops_time_nofirsttimebin):
    ax = axes[i]
    data = sequence_counts_both[seq]
    hl,ll,hh = np.ndarray.tolist(sequence_counts_group[seq].T*group_sizes)[0]
    seq = "$"+seq.replace('Ts','T_2').replace('Tb','T_1').replace('Tc','T_m')+"$"
    ax.text(0,3.05, seq, fontsize=14)
    ax.text(2.0,3.05, "$IG$= {0}, $LH$: {1}, $LL$: {2}, $HH$: {3}".format(str(round(ig,2)),hl,ll,hh), fontsize=10)
    
    heatmap = sns.heatmap(data, ax=ax, cmap=plt.cm.Blues,alpha=0.8, vmin=0, vmax=maximum, cbar=False,
                          annot=True) #, annot_kws={'fontweight':'bold'})
    fig.set_size_inches(10,40)
    ax.set_frame_on(False)
    ax.set_yticks(np.arange(data.shape[0]) + 0.5)
    ylabels = ['$HH$','$LL$','$LH$']
    ax.set_yticklabels(ylabels)
    ax.set_xticklabels(['','','',''])
        
    i += 1